In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cur

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('csv', data_files='/content/converted_prompts_answers2.csv')


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [ ]:
# The model you want to train from Hugging Face Hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The dataset to use (can be customized to your own dataset)
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"


In [ ]:
# QLoRA parameters
lora_r = 64         # LoRA attention dimension
lora_alpha = 16     # Alpha parameter for LoRA scaling
lora_dropout = 0.1  # Dropout probability for LoRA layers


In [ ]:
# bitsandbytes parameters
use_4bit = True  # Use 4-bit precision for the model
bnb_4bit_compute_dtype = "float16"  # Data type for 4-bit model
bnb_4bit_quant_type = "nf4"         # Quantization type
use_nested_quant = False            # Use double quantization for 4-bit models


In [ ]:
# Training arguments parameters
output_dir = "./results"  # Directory for model checkpoints
num_train_epochs = 1      # Number of epochs
per_device_train_batch_size = 4   # Batch size per device (GPU)
per_device_eval_batch_size = 4    # Batch size for evaluation
gradient_accumulation_steps = 1   # Accumulate gradients over multiple steps
gradient_checkpointing = True     # Use gradient checkpointing for memory efficiency
max_grad_norm = 0.3      # Gradient clipping
learning_rate = 2e-4     # Learning rate
weight_decay = 0.001     # Weight decay
optim = "paged_adamw_32bit"  # Optimizer to use
lr_scheduler_type = "cosine"  # Learning rate scheduler type
warmup_ratio = 0.03      # Warmup ratio
group_by_length = True   # Group sequences by length for efficiency
logging_steps = 25       # Log every 25 steps


In [ ]:
# Load dataset
dataset = load_dataset(dataset_name, split="train")


README.md:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

(…)-00000-of-00001-9ad84bb9cf65a42f.parquet:   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Load model with BitsAndBytes configuration for quantization
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}  # Load model on GPU 0
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # To avoid overflow issues


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


In [ ]:
# Define training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=0,  # Set to 0 to disable checkpoint saving (or adjust as needed)
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=False,  # Set to True if you're using FP16 precision
    bf16=False,  # Set to True if you're using BF16 precision (for A100)
    max_grad_norm=max_grad_norm,
    max_steps=-1,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",  # Logs to TensorBoard
)


In [ ]:
# Set up the Supervised Fine-Tuning (SFT) Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",  # Make sure your dataset has a "text" field
    max_seq_length=None,  # Adjust max sequence length if needed
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,  # Set to True to pack short examples into longer sequences
)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Start training
trainer.train()


Step,Training Loss
25,1.272400
50,1.538000
75,1.191800
100,1.478300
125,1.174000
150,1.374100
175,1.142600
200,1.525800
225,1.138200
250,1.444300


TrainOutput(global_step=250, training_loss=1.3279443893432616, metrics={'train_runtime': 1657.4993, 'train_samples_per_second': 0.603, 'train_steps_per_second': 0.151, 'total_flos': 9701518555545600.0, 'train_loss': 1.3279443893432616, 'epoch': 1.0})

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
# Ignore warnings
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "generate visulization fun activity for 6 year old "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=Tr

<s>[INST] generate visulization activity for 6 year old child which should be fun [/INST] Sure, here are some ideas for visualization activities for a 6-year-old child that are fun and engaging:

1. Colorful Clouds: Have the child lie down on a mat or blanket and close their eyes. Encourage them to imagine a big, fluffy cloud above their head. Ask them to imagine the cloud changing colors, like a rainbow. They can also imagine the cloud moving around and changing shapes.
2. Magic Paintbrush: Give the child a paintbrush and some paper. Encourage them to paint a picture of anything they want, like a tree, a cat, or a sunset. As they paint, ask them to imagine that they are painting with a magic brush that can make anything they want appear on the paper.
3. Imaginary Friends: Have
